# MLB Analytics Pipeline

### Necessary Imports

In [159]:
import pandas as pd
pd.set_option('display.max_rows', 200)

import seaborn as sns

# Gather Data

### Function to Web-Scrape Player Game Logs

In [173]:
# Pitchers - ERA is their season avg at that time

def scrape_gamelogs(year, player_id, cat):
    df_player=pd.DataFrame()
    tables = pd.read_html(f'https://www.espn.com/mlb/player/gamelog/_/id/{player_id}/year/{year}/category/{cat}')
    for i in range(len(tables)-1):
        try:
            df_player = df_player.append(tables[i])
        except: break
    df_player = df_player.iloc[::-1].reset_index(drop=True)
    df_player.loc[0,['Date','OPP','Result']] = 'Totals'
    
    try: df_player = df_player.drop(columns=['Regular Season Stats'])
    except: pass
    
    months = ['april','may','june','july','august','september','october','Postseason','Totals']
    df_totals = pd.DataFrame()
    for i in months:
        try:
            df_totals = df_totals.append(df_player.loc[df_player['Date'] == i]).reset_index(drop=True)
            df_player = df_player[df_player.Date != i]
        except: pass
    
    try: df_totals = df_totals.drop(columns=['OPP','Result'])
    except: pass
    
    df_totals.Date = df_totals.Date.str.capitalize()
    
    return df_player, df_totals

df_gonzalez_box, df_gonzalez_totals = scrape_gamelogs(2022, 39400, 'batting')
display(df_gonzalez_totals)
display(df_gonzalez_box)

,Date,AB,R,H,2B,3B,HR,RBI,BB,HBP,SO,SB,CS,AVG,OBP,SLG,OPS
0,May,21,2,9,3,0,0,2,0,0,3,0,0,0.429,0.429,0.571,1.000
1,June,102,10,26,9,0,2,12,5,1,25,0,0,0.255,0.294,0.402,0.696
2,August,98,7,28,8,0,3,9,3,0,23,1,1,0.286,0.307,0.459,0.766
3,September,121,15,36,5,0,6,20,6,2,21,0,1,0.298,0.338,0.488,0.826
4,October,20,5,8,2,0,0,0,1,0,3,0,0,0.400,0.429,0.500,0.929
5,Postseason,31,2,7,0,0,1,4,0,1,9,0,0,0.226,0.250,0.323,0.573
6,Totals,362,39,107,27,0,11,43,15,3,75,1,2,0.296,0.327,0.461,0.789


,Date,OPP,Result,AB,R,H,2B,3B,HR,RBI,BB,HBP,SO,SB,CS,AVG,OBP,SLG,OPS
2,Thu 5/26,@DET,L4-3,4,0,2,1,0,0,0,0,0,0,0,0,0.500,0.500,0.750,1.250
3,Sat 5/28,@DET,W8-1,4,0,2,1,0,0,0,0,0,0,0,0,0.500,0.500,0.750,1.250
4,Sun 5/29,@DET,L2-1,4,0,1,0,0,0,0,0,0,2,0,0,0.417,0.417,0.583,1.000
5,Mon 5/30,vsKC,W7-3,4,1,2,0,0,0,0,0,0,1,0,0,0.438,0.438,0.563,1.000
6,Tue 5/31,vsKC,W8-3,5,1,2,1,0,0,2,0,0,0,0,0,0.429,0.429,0.571,1.000
8,Wed 6/1,vsKC,W4-0,4,0,0,0,0,0,0,0,0,1,0,0,0.360,0.360,0.480,0.840
9,Fri 6/3,@BAL,W6-3,4,0,2,0,0,0,0,0,0,0,0,0,0.379,0.379,0.483,0.862
10,Sat 6/4,@BAL,L5-4,4,1,1,0,0,0,0,0,0,0,0,0,0.364,0.364,0.455,0.818
11,Sun 6/5,@BAL,W3-2,3,0,1,0,0,0,0,1,0,1,0,0,0.361,0.378,0.444,0.823
12,Tue 6/7,vsTEX,W6-3,4,0,2,2,0,0,1,0,0,0,0,0,0.375,0.390,0.500,0.890


### Function to Web-Scrape Player Splits

In [172]:
def scrape_splits(year, player_id, cat):
    df_player=pd.DataFrame()
    tables = pd.read_html(f'https://www.espn.com/mlb/player/splits/_/id/{player_id}/year/{year}/category/{cat}')
    for i in range(len(tables)-1):
        try:
            df_player = pd.concat([df_player, tables[i]], axis=1)
        except: break
            
    df_player.columns = df_player.iloc[0]
    df_player['Cat'] = ''
    cats = ['Overall','Opponent Batting', 'Breakdown','Day / Month','Opponent','Stadium','Position','Count','Batting Order','Situation','Inning Pitches','Rest (as reliever)']
    for i in range(len(df_player)):
        for j in cats:
            if df_player.Overall[i] ==j:
                x=j
        df_player.Cat[i] = x
    
    for i in cats:
        df_player = df_player[df_player.Overall != i]
    
    pivot_player = df_player.groupby(['Cat','Overall']).first()
    
    return df_player, pivot_player

df_clase_split, pivot_clase_split = scrape_splits(2022, 41743, 'pitching')
display(pivot_clase_split)

0                                             ERA   W   L  SV SVOP  GP   GS  \
Cat                Overall                                                    
Batting Order      Batting #1                  26   3   6   1    0   0    3   
                   Batting #2                  26   1   2   0    0   0    0   
                   Batting #3                  27   5   7   0    0   1    5   
                   Batting #4                  25   0   1   1    0   0    0   
                   Batting #5                  27   3   3   1    0   0    3   
                   Batting #6                  31   0   5   0    0   0    0   
                   Batting #7                  37   3  10   1    0   2    3   
                   Batting #8                  28   3   4   2    0   0    3   
                   Batting #9                  31   2   5   0    0   0    1   
Breakdown          Away                      2.34   2   4  25   29  39    0   
                   Day                       1.82   1   2  15   16  26    0   
                   Home                      0.47   1   0  17   17  38    0   
                   Night                     1.13   2   2  27   30  51    0   
                   Right / Left                AB   R   H  2B   3B  HR  RBI   
                   vs. Left                   112   9  20   0    0   2    8   
                   vs. Right                  146  11  23   6    0   1   10   
Count              Count 0-0                   32   5  10   1    0   1    4   
                   Count 0-1                   27   3   9   1    0   1    3   
                   Count 0-2                   60   2   7   3    0   1    2   
                   Count 1-0                   20   2   2   0    0   0    2   
                   Count 1-1                   20   0   0   0    0   0    0   
                   Count 1-2                   52   4   5   1    0   0    4   
                   Count 2-0                    3   1   1   0    0   0    1   
                   Count 2-1                    7   0   1   0    0   0    0   
                   Count 2-2                   26   3   6   0    0   0    2   
                   Count 3-0                    0   0   0   0    0   0    0   
                   Count 3-1                    1   0   0   0    0   0    0   
                   Count 3-2                   10   0   2   0    0   0    0   
Day / Month        April                     4.91   0   2   4    5   9    0   
                   August                    0.79   0   1   7    7  13    0   
                   July                      0.90   0   0   5    5  10    0   
                   June                      0.00   1   0  11   11  15    0   
                   Last 15 Days              1.13   0   0   6    6   8    0   
                   Last 30 Days              2.40   1   1  11   13  16    0   
                   Last 7 Days               0.00   0   0   3    3   3    0   
                   May                       0.82   1   0   3    4  11    0   
                   October                   0.00   0   0   2    2   2    0   
                   September                 2.25   1   1  10   12  17    0   
Inning Pitches     1st Time Faced In Game     258  20  43   6    0   3   18   
                   2nd Time Faced In Game       0   0   0   0    0   0    0   
                   3rd Time Faced In Game       0   0   0   0    0   0    0   
                   4th+ Time Faced In Game      0   0   0   0    0   0    0   
Opponent           vs. ARI                   0.00   0   0   0    0   1    0   
                   vs. BAL                   0.00   0   0   2    2   3    0   
                   vs. BOS                   0.00   0   0   1    1   2    0   
                   vs. CHW                   1.00   2   0   5    7   9    0   
                   vs. CIN                   4.50   0   0   0    0   2    0   
                   vs. COL                   0.00   0   0   3    3   3    0   
                   vs. DET                   1.23 